# This notebook will be used to load a pre-trained model and save the outputs layer by layer

Tensorflow will be used for this endeavor

## sources

[1] VGG16 run: https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/

[2] Layers output: https://stackoverflow.com/questions/41711190/keras-how-to-get-the-output-of-each-layer

[3] Getting layers output: https://github.com/philipperemy/keract#get-activations-nodeslayers-outputs-as-numpy-arrays

Loading imports


In [2]:
# import
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import os 
import numpy as np
from keract import get_activations, display_activations
import csv 


2022-09-20 10:51:31.549454: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Preprocess the data

In [3]:
# load image from path
path = './data/mug.jpg'
image = load_img(path, target_size=(224, 224))
# convert raw pixels image to numpy array 
image = img_to_array(image)
#reshape the tensor for VGG16 input
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#call preprocessing for VGG16
image = preprocess_input(image)


In [4]:
filename = f'./outputs-mug.jpg/0_input_0.csv'
img_matrix = image[0]
 # write to file
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the fields
    csvwriter.writerow(['row col channels (xyc)'])
    # get number of channels
    for x in range(len(img_matrix)): # read rows
        csvwriter.writerow([f'row {x}---'])
        for y in range(len(img_matrix[x])): # read columns
            # for c in range(len(matrix[0 , 0])): # read channels
            csvwriter.writerow([f'col {y}: '] + [c for c in img_matrix[x, y]])

Building VGG model


In [5]:
model = VGG16(
    include_top=True,
    weights='imagenet',
    classes=1000,
    classifier_activation='softmax'
)

print(model.summary())


2022-09-20 10:51:39.168140: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-20 10:51:39.170048: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-20 10:51:39.205029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-20 10:51:39.205427: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 960M computeCapability: 5.0
coreClock: 1.0975GHz coreCount: 5 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 74.65GiB/s
2022-09-20 10:51:39.205460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-09-20 10:51:39.208611: I tensorflow/stream_executor/platform/def

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [17]:
# write the kernels to file
for i, layer in enumerate(model.layers):
    # skip input layer
    if not hasattr(layer, 'kernel'):
        continue
    # get weights
    # weights = layer.get_weights()
    # get name
    name = layer.name
    # write to file
    filename = f'./model/kernels/{name}_kernels.txt'
    # write kernal weights to file
    print(f'kernel {layer.kernel}')
    # tf.print(layer.kernel, output_stream = f"file://{filename}")
    with open(filename, 'w') as f:
        # write the weights to file
        f.write(str(layer.kernel))

kernel <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
array([[[[ 4.29470569e-01,  1.17273867e-01,  3.40129584e-02, ...,
          -1.32241577e-01, -5.33475243e-02,  7.57738389e-03],
         [ 5.50379455e-01,  2.08774377e-02,  9.88311544e-02, ...,
          -8.48205537e-02, -5.11389151e-02,  3.74943428e-02],
         [ 4.80015397e-01, -1.72696680e-01,  3.75577137e-02, ...,
          -1.27135560e-01, -5.02991639e-02,  3.48965675e-02]],

        [[ 3.73466998e-01,  1.62062630e-01,  1.70863140e-03, ...,
          -1.48207128e-01, -2.35300660e-01, -6.30356818e-02],
         [ 4.40074533e-01,  4.73412387e-02,  5.13819456e-02, ...,
          -9.88498852e-02, -2.96195745e-01, -7.04357103e-02],
         [ 4.08547401e-01, -1.70375049e-01, -4.96297423e-03, ...,
          -1.22360572e-01, -2.76450396e-01, -3.90796512e-02]],

        [[-6.13601133e-02,  1.35693997e-01, -1.15694344e-01, ...,
          -1.40158370e-01, -3.77666801e-01, -3.00509870e-01],
         [-8.1

In [18]:

# write the weights to file
for layer in model.layers:
    # get weights
    weights = layer.get_weights()
    print(f'layer {name} dtype: {weights[0][0][0][0][0].dtype}')
    # get name
    name = layer.name
    # write to file
    filename = f'./model/{name}_weights.csv'
    with open(filename, 'w') as csvfile:
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)
        # writing the fields
        csvwriter.writerow(['weights'])
        # writing the data rows
        # print weights directory
        
        csvwriter.writerows(weights)
        

2022-09-19 19:07:12.844185: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 411041792 exceeds 10% of free system memory.


Running model

In [ ]:
# read the output logits from VGG16 on our mug input
logits = model.predict(image)
# convert logits to class labels 
labels = decode_predictions(logits, top=1000)
# get highest confidence label
label = labels[0][0]
# print classification 
print('%s (%.2f%%)' % (label[1], label[2]*100))

Capture Layers' output


In [ ]:
# call to fetch the activations of the model.
activations = get_activations(model, image, auto_compile=True)
activations

In [ ]:
label_pairs = []
# read output classes
with open("./data/imagenet1000_clsidx_to_labels.txt", "r") as f:
    for line in f:
        label_idx, label_val = line.split(':')
        label_idx = int(label_idx)
        label_val = label_val.split(',')[0]
        label_val = label_val.strip(' \'')
        label_pairs.append((label_idx, label_val))
        
# print(label_pairs)

In [ ]:

# print the activations shapes.
from fileinput import filename


[print(k, '->', v.shape, '- Numpy array') for (k, v) in activations.items()]

# write activations to files
index = 0
for layer, matrix in activations.items():
    # layer filename
    filename = f"./outputs-mug.jpg/{index}_{layer}.csv"
    matrix = matrix[0]

    if index < 19: # shape length is 4
        # write to file
        with open(filename, 'w') as csvfile:
            # creating a csv writer object
            csvwriter = csv.writer(csvfile)
            # writing the fields
            csvwriter.writerow(['row col channels (xyc)'])
            # get number of channels
            for x in range(len(matrix)): # read rows
                csvwriter.writerow([f'row {x}---'])
                for y in range(len(matrix[0])): # read columns
                    # for c in range(len(matrix[0 , 0])): # read channels
                    csvwriter.writerow([f'col {y}: '] + [c for c in matrix[x, y]])


    
    elif index < 22: # shape length is 2
        values = matrix
        # write to file
        with open(filename, 'w') as csvfile:
            # creating a csv writer object
            csvwriter = csv.writer(csvfile)
            # writing the fields
            csvwriter.writerow(['id', 'value'])
            # writing the data rows
            for idx, value in enumerate(values):
                csvwriter.writerow([idx, value])
        
    elif index == 22: # final output
        outputs = matrix
        with open(filename, 'w') as csvfile:
            # creating a csv writer object
            csvwriter = csv.writer(csvfile)
            # writing the fields
            csvwriter.writerow(['classes', 'probabilities'])
            
            # writing the data rows
            for idx, pair in enumerate(label_pairs):
                csvwriter.writerow([label_pairs[idx][1], outputs[idx]])

        
    index += 1
    


In [ ]:

# display activations
display_activations(activations, cmap=None, save=False, directory='.', data_format='channels_last', fig_size=(24, 24), reshape_1d_layers=False)